A Fórmula Mágica é um modelo de fator que classifica as ações por dois fatores:

1. O valor de uma empresa em relação aos seus lucros determina quão “barato” é o preço de mercado de uma ação. Greenblatt define “Barato” como o valor de uma empresa em relação aos seus lucros. Na maioria das vezes, podemos ver isto representado pelo P/L, Greenblatt prefere olhar para EV/EBIT. Isto permite que empresas com diferentes estruturas de endividamento e impostos sejam comparadas mais facilmente.

2. O retorno sobre o capital determina o quão “boa” é uma empresa. “Bom” é representado pelo ROIC, Greenblatt quantifica a quantidade de capital tangível necessária para operar um negócio e quanto dinheiro cada unidade de capital aplicado irá render.

In [1]:
earning_yield = "EV/EBIT"
return_on_capital = "ROIC"

In [2]:
import warnings
warnings.filterwarnings('ignore')
# from datetime import datetime, date
import numpy as np
import pandas as pd
# import yfinance as yf

def pct_to_float(number):
    """Convert string to float, remove % char and set decimal point to '.'."""
    return float(number.strip("%").replace(".", "").replace(",", "."))

dados do site https://www.fundamentus.com.br/resultado.php

In [3]:
import requests
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
url = 'https://www.fundamentus.com.br/resultado.php'
#junta com a requests
r = requests.get(url, headers=header)
# read_html do pandas põe a tabela num dataframe
funda = pd.read_html(r.text, index_col="Papel",
                     decimal=',', thousands='.',encoding='ISO-8859-1', 
                     converters={'ROE': pct_to_float,
                                 'ROIC': pct_to_float,
                                 'Div.Yield':pct_to_float,
                                 'Mrg Ebit':pct_to_float,
                                 'Mrg. Líq.':pct_to_float,
                                 'Cresc. Rec.5a':pct_to_float,
                                 },
)[0]
dfunds = pd.DataFrame(funda)
dfunds.tail(5)

,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
Papel,,,,,,,,,,,,,,,,,,,,
UBBR4,7.49,610.27,1.99,0.000,0.0,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.33,0.0,1.031720e+10,0.00,10.58
UBBR11,14.75,1201.81,3.91,0.000,0.0,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.33,0.0,1.031720e+10,0.00,10.58
CEAB3,9.69,1279.18,0.99,0.445,0.0,0.317,2.41,5.41,-1.52,6.03,2.57,8.22,0.03,1.39,8.47,0.08,28625700.0,3.012720e+09,0.56,9.36
UBBR3,18.00,1466.61,4.77,0.000,0.0,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.33,0.0,1.031720e+10,0.00,10.58
CEPE3,128.00,1591.72,6.04,1.127,0.0,0.695,29.30,7.83,-1.13,13.88,10.37,14.40,0.07,1.10,10.01,0.38,0.0,1.580000e+09,5.15,331.55


In [4]:
dfunds.columns

Index(['Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo', 'P/Cap.Giro',
       'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA', 'Mrg Ebit',
       'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses', 'Patrim. Líq',
       'Dív.Brut/ Patrim.', 'Cresc. Rec.5a'],
      dtype='object')

filtros e ranking

In [5]:
funds = dfunds.copy()

funds =  funds[funds[earning_yield] > 0]
funds =  funds[funds[return_on_capital] > 0]

funds =  funds[funds['Liq.2meses'] > 300000] #Volume diário médio negociado nos últimos 2 meses
funds =  funds[funds['P/L'] > 0] 
# funds =  funds[(funds['P/L'] > 0) & (funds['P/L'] < 60)] 
funds =  funds[funds['Dív.Brut/ Patrim.'] < 4]
# funds =  funds[funds['Cresc. Rec.5a'] > 0]
# funds =  funds[funds['ROE'] > 0]
# funds =  funds[funds['EV/EBITDA'] > 0]


""" magic formula rank."""
funds["Rank_earnings_yield"]   = funds[earning_yield].rank(ascending=True, method="min")
funds["Rank_return_on_capital"]= funds[return_on_capital].rank(ascending=False, method="min")
funds["Rank_Final"] = (funds["Rank_earnings_yield"] + funds["Rank_return_on_capital"])
funds.sort_values(by="Rank_Final", ascending=True, inplace=True)
funds.reset_index(inplace=True)
funds.index = funds.index + 1
# print(dfunds.head(15).to_string())
funds.head(30)

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
1,PSSA3,28.61,8.16,1.49,0.571,4.93,0.418,5.97,0.69,-3.50,...,1.13,83.01,18.25,4.895980e+07,1.241530e+10,0.00,16.93,1.0,1.0,2.0
2,PETR4,40.18,3.83,1.36,0.977,18.04,0.511,-72.19,2.18,-1.07,...,0.95,25.76,35.47,1.404170e+09,3.860070e+11,0.79,20.27,6.0,14.0,20.0
3,PETR3,41.21,3.93,1.39,1.002,17.59,0.524,-74.04,2.23,-1.10,...,0.95,25.76,35.47,4.271320e+08,3.860070e+11,0.79,20.27,7.0,14.0,21.0
4,WIZC3,6.35,10.44,2.21,0.967,7.01,0.443,-17.43,2.22,-1.23,...,0.90,24.37,21.18,2.575630e+06,4.592540e+08,0.77,11.69,5.0,17.0,22.0
5,CAMB3,11.62,6.40,2.24,1.026,2.56,1.387,5.94,4.43,13.49,...,1.93,33.41,35.03,6.950300e+05,2.192080e+08,0.13,28.07,19.0,5.0,24.0
6,UNIP3,63.00,8.44,2.33,1.150,5.02,1.056,5.14,5.12,-9.18,...,1.92,29.80,27.61,8.493120e+05,2.808340e+09,0.47,21.79,20.0,9.0,29.0
7,CMIN3,6.02,10.72,2.89,1.904,12.91,1.115,3.70,5.64,-7.01,...,2.96,33.73,26.92,4.594610e+07,1.144420e+10,0.74,-9.63,27.0,3.0,30.0
8,VLID3,18.97,7.67,1.14,0.791,5.04,0.579,2.30,3.38,25.11,...,2.00,22.55,14.90,9.373300e+06,1.358760e+09,0.54,1.44,11.0,20.0,31.0
9,UNIP6,66.86,8.96,2.47,1.221,5.21,1.120,5.46,5.44,-9.75,...,1.92,29.80,27.61,1.074480e+07,2.808340e+09,0.47,21.79,26.0,9.0,35.0
10,VALE3,66.88,7.60,1.59,1.459,9.09,0.666,15.50,3.56,-1.82,...,1.28,20.67,20.91,1.841090e+09,1.909650e+11,0.35,7.85,12.0,25.0,37.0


retira bancos

In [6]:
setor=20
url = f'http://www.fundamentus.com.br/resultado.php?setor={setor}'
hdr = {'User-agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201' ,
           'Accept': 'text/html, text/plain, text/css, text/sgml, */*;q=0.01' ,
           'Accept-Encoding': 'gzip, deflate' ,
           }
content = requests.get(url, headers=hdr)
df = pd.read_html(content.text, decimal=",", thousands='.')[0]
bancos = list(df['Papel'])

funds = funds[~funds['Papel'].isin(bancos)]
funds.head(30)

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
1,PSSA3,28.61,8.16,1.49,0.571,4.93,0.418,5.97,0.69,-3.50,...,1.13,83.01,18.25,4.895980e+07,1.241530e+10,0.00,16.93,1.0,1.0,2.0
2,PETR4,40.18,3.83,1.36,0.977,18.04,0.511,-72.19,2.18,-1.07,...,0.95,25.76,35.47,1.404170e+09,3.860070e+11,0.79,20.27,6.0,14.0,20.0
3,PETR3,41.21,3.93,1.39,1.002,17.59,0.524,-74.04,2.23,-1.10,...,0.95,25.76,35.47,4.271320e+08,3.860070e+11,0.79,20.27,7.0,14.0,21.0
4,WIZC3,6.35,10.44,2.21,0.967,7.01,0.443,-17.43,2.22,-1.23,...,0.90,24.37,21.18,2.575630e+06,4.592540e+08,0.77,11.69,5.0,17.0,22.0
5,CAMB3,11.62,6.40,2.24,1.026,2.56,1.387,5.94,4.43,13.49,...,1.93,33.41,35.03,6.950300e+05,2.192080e+08,0.13,28.07,19.0,5.0,24.0
6,UNIP3,63.00,8.44,2.33,1.150,5.02,1.056,5.14,5.12,-9.18,...,1.92,29.80,27.61,8.493120e+05,2.808340e+09,0.47,21.79,20.0,9.0,29.0
7,CMIN3,6.02,10.72,2.89,1.904,12.91,1.115,3.70,5.64,-7.01,...,2.96,33.73,26.92,4.594610e+07,1.144420e+10,0.74,-9.63,27.0,3.0,30.0
8,VLID3,18.97,7.67,1.14,0.791,5.04,0.579,2.30,3.38,25.11,...,2.00,22.55,14.90,9.373300e+06,1.358760e+09,0.54,1.44,11.0,20.0,31.0
9,UNIP6,66.86,8.96,2.47,1.221,5.21,1.120,5.46,5.44,-9.75,...,1.92,29.80,27.61,1.074480e+07,2.808340e+09,0.47,21.79,26.0,9.0,35.0
10,VALE3,66.88,7.60,1.59,1.459,9.09,0.666,15.50,3.56,-1.82,...,1.28,20.67,20.91,1.841090e+09,1.909650e+11,0.35,7.85,12.0,25.0,37.0


retirar seguradoras

In [7]:
setor=31
url = f'http://www.fundamentus.com.br/resultado.php?setor={setor}'
hdr = {'User-agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201' ,
           'Accept': 'text/html, text/plain, text/css, text/sgml, */*;q=0.01' ,
           'Accept-Encoding': 'gzip, deflate' ,
           }
content = requests.get(url, headers=hdr)
df = pd.read_html(content.text, decimal=",", thousands='.')[0]
seguros = list(df['Papel'])

funds = funds[~funds['Papel'].isin(seguros)]
funds.head(30)

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
2,PETR4,40.18,3.83,1.36,0.977,18.04,0.511,-72.19,2.18,-1.07,...,0.95,25.76,35.47,1.404170e+09,3.860070e+11,0.79,20.27,6.0,14.0,20.0
3,PETR3,41.21,3.93,1.39,1.002,17.59,0.524,-74.04,2.23,-1.10,...,0.95,25.76,35.47,4.271320e+08,3.860070e+11,0.79,20.27,7.0,14.0,21.0
5,CAMB3,11.62,6.40,2.24,1.026,2.56,1.387,5.94,4.43,13.49,...,1.93,33.41,35.03,6.950300e+05,2.192080e+08,0.13,28.07,19.0,5.0,24.0
6,UNIP3,63.00,8.44,2.33,1.150,5.02,1.056,5.14,5.12,-9.18,...,1.92,29.80,27.61,8.493120e+05,2.808340e+09,0.47,21.79,20.0,9.0,29.0
7,CMIN3,6.02,10.72,2.89,1.904,12.91,1.115,3.70,5.64,-7.01,...,2.96,33.73,26.92,4.594610e+07,1.144420e+10,0.74,-9.63,27.0,3.0,30.0
8,VLID3,18.97,7.67,1.14,0.791,5.04,0.579,2.30,3.38,25.11,...,2.00,22.55,14.90,9.373300e+06,1.358760e+09,0.54,1.44,11.0,20.0,31.0
9,UNIP6,66.86,8.96,2.47,1.221,5.21,1.120,5.46,5.44,-9.75,...,1.92,29.80,27.61,1.074480e+07,2.808340e+09,0.47,21.79,26.0,9.0,35.0
10,VALE3,66.88,7.60,1.59,1.459,9.09,0.666,15.50,3.56,-1.82,...,1.28,20.67,20.91,1.841090e+09,1.909650e+11,0.35,7.85,12.0,25.0,37.0
11,LEVE3,36.36,6.87,3.14,1.110,25.73,1.565,5.08,6.04,10.74,...,1.91,36.02,45.73,1.992170e+07,1.568530e+09,0.31,18.50,36.0,2.0,38.0
12,DEXP3,12.61,7.43,1.72,0.658,3.10,0.835,1.86,5.80,3.52,...,3.17,22.52,23.11,1.371720e+06,6.905010e+08,0.42,28.46,25.0,21.0,46.0


retirar BDR

In [8]:
# Padrão a ser buscado XYZW3[2,3,4,5]
padrao = r'[A-Z]{4}3[2-5]'

# Filtrando as linhas onde o padrão ocorre na coluna 'Papel'
funds = funds[~funds['Papel'].str.contains(padrao)]
funds.head(30)

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
2,PETR4,40.18,3.83,1.36,0.977,18.04,0.511,-72.19,2.18,-1.07,...,0.95,25.76,35.47,1.404170e+09,3.860070e+11,0.79,20.27,6.0,14.0,20.0
3,PETR3,41.21,3.93,1.39,1.002,17.59,0.524,-74.04,2.23,-1.10,...,0.95,25.76,35.47,4.271320e+08,3.860070e+11,0.79,20.27,7.0,14.0,21.0
5,CAMB3,11.62,6.40,2.24,1.026,2.56,1.387,5.94,4.43,13.49,...,1.93,33.41,35.03,6.950300e+05,2.192080e+08,0.13,28.07,19.0,5.0,24.0
6,UNIP3,63.00,8.44,2.33,1.150,5.02,1.056,5.14,5.12,-9.18,...,1.92,29.80,27.61,8.493120e+05,2.808340e+09,0.47,21.79,20.0,9.0,29.0
7,CMIN3,6.02,10.72,2.89,1.904,12.91,1.115,3.70,5.64,-7.01,...,2.96,33.73,26.92,4.594610e+07,1.144420e+10,0.74,-9.63,27.0,3.0,30.0
8,VLID3,18.97,7.67,1.14,0.791,5.04,0.579,2.30,3.38,25.11,...,2.00,22.55,14.90,9.373300e+06,1.358760e+09,0.54,1.44,11.0,20.0,31.0
9,UNIP6,66.86,8.96,2.47,1.221,5.21,1.120,5.46,5.44,-9.75,...,1.92,29.80,27.61,1.074480e+07,2.808340e+09,0.47,21.79,26.0,9.0,35.0
10,VALE3,66.88,7.60,1.59,1.459,9.09,0.666,15.50,3.56,-1.82,...,1.28,20.67,20.91,1.841090e+09,1.909650e+11,0.35,7.85,12.0,25.0,37.0
11,LEVE3,36.36,6.87,3.14,1.110,25.73,1.565,5.08,6.04,10.74,...,1.91,36.02,45.73,1.992170e+07,1.568530e+09,0.31,18.50,36.0,2.0,38.0
12,DEXP3,12.61,7.43,1.72,0.658,3.10,0.835,1.86,5.80,3.52,...,3.17,22.52,23.11,1.371720e+06,6.905010e+08,0.42,28.46,25.0,21.0,46.0


In [9]:
funds

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
2,PETR4,40.18,3.83,1.36,0.977,18.04,0.511,-72.19,2.18,-1.07,...,0.95,25.76,35.47,1.404170e+09,3.860070e+11,0.79,20.27,6.0,14.0,20.0
3,PETR3,41.21,3.93,1.39,1.002,17.59,0.524,-74.04,2.23,-1.10,...,0.95,25.76,35.47,4.271320e+08,3.860070e+11,0.79,20.27,7.0,14.0,21.0
5,CAMB3,11.62,6.40,2.24,1.026,2.56,1.387,5.94,4.43,13.49,...,1.93,33.41,35.03,6.950300e+05,2.192080e+08,0.13,28.07,19.0,5.0,24.0
6,UNIP3,63.00,8.44,2.33,1.150,5.02,1.056,5.14,5.12,-9.18,...,1.92,29.80,27.61,8.493120e+05,2.808340e+09,0.47,21.79,20.0,9.0,29.0
7,CMIN3,6.02,10.72,2.89,1.904,12.91,1.115,3.70,5.64,-7.01,...,2.96,33.73,26.92,4.594610e+07,1.144420e+10,0.74,-9.63,27.0,3.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,LOGG3,21.93,11.66,0.60,10.176,3.18,0.358,5.54,13.87,-1.41,...,1.77,2.79,5.15,4.810220e+06,3.729940e+09,0.51,16.33,155.0,152.0,307.0
157,NGRD3,1.12,28.85,0.59,0.987,1.32,0.398,1.61,55.56,3.79,...,2.37,1.11,2.03,5.144120e+05,4.564490e+08,0.11,7.00,150.0,160.0,310.0
158,MELK3,4.86,13.86,0.86,0.992,11.54,0.412,0.75,27.45,1.24,...,3.08,1.89,6.17,6.549770e+05,1.171940e+09,0.24,21.63,154.0,158.0,312.0
159,JALL3,7.50,104.66,1.10,1.358,5.73,0.338,1.21,21.33,-1.06,...,3.88,1.94,1.05,4.747800e+06,2.072120e+09,1.94,9.80,159.0,157.0,316.0


A fórmula:
1. Estabeleça uma capitalização de mercado mínima.
2. Exclua ações de serviços públicos e financeiras  (ou seja, fundos mútuos, bancos e companhias de seguros).
3. Excluir empresas estrangeiras (ADRs)
4. Determine o rendimento dos lucros da empresa = EBIT/EV
5. Determine o retorno sobre o capital da empresa = EBIT/ (Ativo Fixo Líquido + Capital de Giro).
6. Classifique todas as empresas acima da capitalização de mercado escolhida pelo maior rendimento de lucros e maior retorno sobre o capital (classificado como porcentagens)
7. Invista em 20 a 30 empresas com melhor classificação, acumulando 2 a 3 posições por mês durante um período de 12 meses
8. Reequilibre o portfólio uma vez por ano, vendendo os perdedores uma semana antes do ano e os vencedores uma semana depois do ano.
9. Continuar por um período de longo prazo (5 a 10+ anos)

https://en.wikipedia.org/wiki/Magic_formula_investing

O retorno sobre o capital de Greenblatt difere de um valor típico de ROE ou ROIC. Dentro da Fórmula Mágica, o retorno sobre o capital de uma empresa é medido como EBIT/capital tangível empregado. Em outras palavras, estamos tentando encontrar os custos tangíveis para o negócio na geração dos lucros reportados dentro do período, onde o capital tangível empregado é definido mais precisamente como Capital Circulante Líquido mais Ativos Fixos Líquidos.

O capital de giro líquido é simplesmente o total do ativo circulante menos o passivo circulante, com um ajuste para remover dívidas com juros de curto prazo do passivo circulante e outro para remover o excesso de caixa. Greenblatt não oferece detalhes sobre como o excesso de caixa deve ser considerado, mas muitas vezes é calculado com base em uma porcentagem do caixa necessário em relação às vendas geradas em um período.